## 리팩토링한 코드를 바탕으로 데이터를 전처리하고 prophet 라이브러리를 활용하는 프레임입니다.

#### 1. 모듈과 데이터를 불러오고 전처리합니다.

In [1]:
import os, sys
path_name= os.path.dirname(os.path.abspath(os.path.dirname(os.getcwd())))
sys.path.append(path_name)
print(path_name)

C:\Nextop\nextop-engine\nextop_engine


In [2]:
from _element import feature_control as ft_c
from _element.data_control import Data, DataRestruction, DataAddition
from _element import varr
from _element import calculations as calc
from _element.result_control import Result

from _alg.prophet import Prophet_timeseries
from _alg.arima import Arima_timeseries

from _evaluation.cross_validation import Cross_Validation
from _evaluation import compare

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture

In [4]:
INPUT_FILENAME= 'KPP일별투입(10_17)_withallobj.xlsx'
df_raw= ft_c.xlsx_opener(path_name+varr.DF_DIR, INPUT_FILENAME)

In [5]:
data= Data()
data.setRawData(df_raw)

          ds  1025  1041  1057  1091  1111  1117  1119  1127  1163  ...    \
0 2010-01-01     0     0     0     0     0     0     0     0     0  ...     
1 2010-01-02   192  6260     0     0     0     0     0     0     0  ...     
2 2010-01-03     0     0     0     0     0     0     0     0     0  ...     
3 2010-01-04     0  5141     0   320     0     0     0     0     0  ...     
4 2010-01-05   130  5494     0  1186     0     0     0     0     0  ...     

   1754  1797  1800   1815  1817  1852  1853  1878  1891  y_sum  
0     0     0     0   2652     0     0     0     0     0   2662  
1   337     0     0  26443     0     0   459   626     0  39352  
2     0     0     0    404     0     0     0     0     0    404  
3   154   576     0  22924     0     0  3445  3305     0  44906  
4   367   294     0  39081   320     0  3861  4362   238  69481  

[5 rows x 36 columns]
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2891 entries, 0 to 2890
Data columns (total 36 columns):
ds       28

* 1041 유형만을 가져옵니다.

In [6]:
restruct= DataRestruction(data)

In [7]:
restruct.selectSpecificY([1041])

In [8]:
df_commit= restruct.commitRestructedData()

   1041         ds
0     0 2010-01-01
1  6260 2010-01-02
2     0 2010-01-03
3  5141 2010-01-04
4  5494 2010-01-05
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2891 entries, 0 to 2890
Data columns (total 2 columns):
1041    2891 non-null int64
ds      2891 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(1)
memory usage: 67.8 KB
None


* 기상 데이터를 추가합니다.

In [10]:
df_temp= ft_c.xlsx_opener(path_name+varr.TEMP_DATA_DIR, 'temp_data_merged.xlsx')

In [11]:
addition= DataAddition(data)

In [12]:
addition.addXData(df_temp)
df_commit= addition.commitAddedData()

   1041         ds   temp_max   temp_min  rain_amount
0  5698 2010-07-01  30.500000  24.900000          0.0
1  2740 2010-07-02  25.799999  22.700001         70.0
2  2742 2010-07-03  25.600000  22.600000          1.0
3     0 2010-07-04  29.700001  23.100000          0.0
4  3451 2010-07-05  30.600000  21.799999          0.0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2710 entries, 0 to 2709
Data columns (total 5 columns):
1041           2710 non-null int64
ds             2710 non-null datetime64[ns]
temp_max       2710 non-null float64
temp_min       2710 non-null float64
rain_amount    2710 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(1)
memory usage: 127.0 KB
None


In [13]:
data.showData()

   1041         ds   temp_max   temp_min  rain_amount
0  5698 2010-07-01  30.500000  24.900000          0.0
1  2740 2010-07-02  25.799999  22.700001         70.0
2  2742 2010-07-03  25.600000  22.600000          1.0
3     0 2010-07-04  29.700001  23.100000          0.0
4  3451 2010-07-05  30.600000  21.799999          0.0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2710 entries, 0 to 2709
Data columns (total 5 columns):
1041           2710 non-null int64
ds             2710 non-null datetime64[ns]
temp_max       2710 non-null float64
temp_min       2710 non-null float64
rain_amount    2710 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(1)
memory usage: 127.0 KB
None


* 월~토 데이터와 일요일 데이터를 분리합니다.

In [14]:
restruct= DataRestruction(data)

In [15]:
restruct.addRegion(7, 'weekdayRegion', start_num= 3)

In [16]:
condition_only_sun= {
    'weekday': (0,1,2,3,4,5,),
    'sun': (6, )
}
restruct.dividebyTupleCondition(
                                'weekdayRegion',
                                condition_only_sun
                                )

In [17]:
restruct.deleteRegion('weekdayRegion')

In [18]:
# restruct.slicebyTrainTestStructure(1041, ['ds', 'temp_max', 'temp_min', 'rain_amount'])

In [19]:
restruct.commitRestructedData()

   1041         ds   temp_max   temp_min  rain_amount
0  5698 2010-07-01  30.500000  24.900000          0.0
1  2740 2010-07-02  25.799999  22.700001         70.0
2  2742 2010-07-03  25.600000  22.600000          1.0
4  3451 2010-07-05  30.600000  21.799999          0.0
5  6011 2010-07-06  31.000000  22.400000          0.0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2323 entries, 0 to 2709
Data columns (total 5 columns):
1041           2323 non-null int64
ds             2323 non-null datetime64[ns]
temp_max       2323 non-null float64
temp_min       2323 non-null float64
rain_amount    2323 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(1)
memory usage: 108.9 KB
None
    1041         ds   temp_max   temp_min  rain_amount
3      0 2010-07-04  29.700001  23.100000          0.0
10     0 2010-07-11  25.900000  21.000000          0.0
17     0 2010-07-18  29.000000  24.700001          0.0
24     0 2010-07-25  29.600000  23.000000         13.0
31     0 2010-08-01  30.70000

* Train, Test 데이터를 만듭니다.

In [20]:
data.slicebyTrainTestStructure(y= 1041)

#### 2. prophet 라이브러리를 통해 물동량을 예측합니다.

In [21]:
doprophet= Prophet_timeseries(dataclass= data)

In [22]:
doprophet.addModel('weekday,object', 'day', regressor= 'rain_amount')

In [23]:
doprophet.fit('weekday,object')

In [24]:
r= Result()

In [25]:
doprophet.forecast('weekday,object', r.forecast_dict)

C:\Nextop\nextop-engine\nextop_engine\_element\feature_control.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['ds'], box=True, format= '%Y/%m/%d', exact=True


In [30]:
r.forecast_dict

OrderedDict([(frozenset({'weekday,object'}),
              {'MAPE': inf,
               'MAPE_div_std': 97.786720816442042,
               'MAPE_with_std': 34.168459480560792,
               'RMSE': 1148.1701949094002,
               'forecast':       y         ds  temp_max  temp_min  rain_amount         yhat
               0  8733 2017-11-24       2.9      -2.5          0.1  6336.095657
               1  5100 2017-11-25       6.4       0.8         15.7  4818.736617
               0     0 2017-11-26       6.4      -2.4          0.0     8.595435
               2  4866 2017-11-27       8.1       1.8          0.0  4852.865691
               3  5723 2017-11-28       9.8       4.2          1.2  5324.013891
               4  6732 2017-11-29       5.7      -3.2          0.0  5621.200118
               5  6909 2017-11-30       1.8      -7.1          0.0  5491.082030,
               'sMAPE': 15.961297283474392})])

In [27]:
r.mergeResult(by='concat', m_name='weekday,object')

In [28]:
r.calcErrRate()

C:\Nextop\nextop-engine\nextop_engine\_element\calculations.py:13: RuntimeWarning: divide by zero encountered in true_divide
  return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


In [29]:
r.printErrRate()

RMSE with segment None: 1148.1702
MAPE with segment None: inf
MAPE_with_std with segment None: 34.1685
MAPE_div_std with segment None: 97.7867
sMAPE with segment None: 15.9613


